# Isentropic Flow

This Notebook illustrates the use of the *Isentropic Flow* class from **pyTurb**. An *Isentropic Flow* is considered to be *reversible* and to have no disipation, nor friction, maintaining a constant level of entropy. If the fluid undergoes such a transformation, then the energy balance of the fluid allows the transport of kinetic energy to thermal/reversible work.

- pyturb
    - gas_models
        - thermo_prop
        - PerfectIdealGas
        - SemiperfectIdealGas
        - GasMixture
        - IsentropicFlow
        
```python
from pyturb.gas_models import GasMixture
from pyturb.gas_models import PerfectIdealGas
from pyturb.gas_models import SemiperfectIdealGas
from pyturb.gas_models import GasMixture
```


In [3]:
from pyturb.gas_models import IsentropicFlow
from pyturb.gas_models import PerfectIdealGas

In [7]:
air = PerfectIdealGas('Air')
isent = IsentropicFlow(air)
T = 288.15 #K
p = 101325 #Pa
rho = p/air.Rg/T
v = 100 # m/s

a = isent.sound_speed(T)
M = isent.mach_number(v, T)
print('M=', M, 'a=', a)

M= 0.29386738560632114 a= 340.28954861280454


In [6]:

print('Tt_T=', isent.stagnation_static_rel(M))
print('Tt=', isent.stag_temp_from_mach(M, T))
print('Tt=', isent.stag_temp_from_vel(v, T))
print('pt=', isent.stag_pressure_from_mach(M, p))
print('qi=', isent.impact_pressure_from_mach(M, p))
print('qi=', isent.dynamic_pressure(v, rho))
print('v=', isent.vel_from_stag_temp(isent.stag_temp_from_mach(M, T), T))
print('v=', isent.vel_from_mach(M, T))
print('rho=', rho, 'rhot=', isent.stag_density_from_mach(M, rho))
print('T=', isent.stat_temp_from_mach(M, isent.stag_temp_from_mach(M, T)))
print('p=', isent.stat_pressure_from_mach(M, isent.stag_pressure_from_mach(M, p)))


Tt_T= 1.0172705758891332
Tt= 293.1265164424537
Tt= 293.1265164424537
pt= 107583.43674314614
qi= 6258.436743146143
qi= 6125.05531983517
v= 100.0
v= 100.0
rho= 1.225011063967034 rhot= 1.2785930207767764
T= 288.15
p= 101325.0
